# Building your first Enclave Service with Oblivious

Welcome back - hope you've had fun with the differential privacy tutorial. Now we'll be moving on to using secure enclaves to provide a form of input privacy for us and if we have time at the end, your goal will be to combine the two.

#### Note: This notebook is not a notebook.... you are not meant to run it inline, simply meant to share code snippets.

In this tutorial, we'll take advantage of the FastAPI library in Python. If you are familiar with any other language and framework, feel free to just replace the code samples in this section with your prefered alternative.

### Create a new GitHub repo with a few files

First, create a new git repository with GitHub. We'll need to add the following files:

```
├── api.py 
├── uvicorn_serve.py
├── requirements.txt 
├── Dockerfile 
├── README.md (optional)
└── .gitignore (optional)
```

### Code up the App

Next, we'll fill in the code just like we would do if we were planning to run on a normal server.

#### A Simple Server

Next, we can write the simplest of FastAPI servers:

In [ ]:
# api.py

from fastapi import FastAPI, Header
from typing import Optional

import psutil

app = FastAPI()


@app.get("/")
async def root(x_oblv_user_name: Optional[str] = Header(None)):
    return {"message": "Hello " + x_oblv_user_name}

@app.get("/resources")
async def resources(x_oblv_user_name: Optional[str] = Header(None)):
    return {
        "CPU %": psutil.cpu_percent(interval=1, percpu=True),
        "Virtual Memory:": psutil.virtual_memory().available * 100 / psutil.virtual_memory().total
        }

@app.get("/select_model")
async def select_model(model_meta, x_oblv_user_name: Optional[str] = Header(None)):
    return {"model_meta": model_meta}

The only thing that may look unfamiliar from a standard FastAPI server is that we are reading the optional header field `X-OBLV-User-Name` which gets autocast to be snake case (lowercase with underscores) as x_oblv_user_name. When we use PKI pre-shared keys (PSK) with field is added by the proxy so the application can know who it was who made the request.

The simple server above will reply to a request by responding `Hello` to the user's name.


Next, let's write a uvicorn server to launch the application:

In [ ]:
# uvicorn.py

import os
import uvicorn

from api import *

if __name__ == "__main__":
    os.chdir("/code/")
    uvicorn.run("api:app", host="0.0.0.0", port=80, log_level="info")

### Requirements

In the `requirements.txt` file, add the requirements for a basic FastAPI server:


In [ ]:
fastapi>=0.68.0,<0.69.0
pydantic>=1.8.0,<2.0.0
uvicorn>=0.15.0,<0.16.0

### Dockerfile

As a last step, let's write a simple `Dockerfile` to package the api. It should look something like this:

In [ ]:
FROM python:3.8
 
WORKDIR /code
 
COPY ./requirements.txt /code/requirements.txt
 
RUN pip install --no-cache-dir --upgrade -r /code/requirements.txt
 
COPY ./ /code/

CMD ["python", "uvicorn_serve.py"]


To build and run the above packaged application we can simply do the following:

In [ ]:
docker build -t example .  

docker run -p 1234:80 -t example 

### Quick Test

The above should make a simple api when we build and deploy it with docker. To test it from the command line you can use a curl command as follows:


In [ ]:
curl --header "X-OBLV-User-Name: Bob" localhost:1234 

### <<< Now push to GitHub 🙌 >>>

## Creating a Service

Now that we've a simple API working locally, let's make it into a service. The steps are:

Log into the Oblivious console.
Connect your repository and specify it's behaviour.
Add a slightly modified Dockerfile & service executables to launch at runtime
Log into the Oblivious console​
To log into the Oblivious console with GitHub, simply go to https://console.oblivious.ai and hit the GitHub icon (highlighted in red below):

![](https://docs.oblivious.ai/assets/images/login-2d4e59333b35cdc805084161e6ab5806.png)

All of your repositories will now be in the `Repositories` folder. Usese the search feature to find `my-first-oblivious-app` (or whatever you called it) and click on the title.

![](https://docs.oblivious.ai/assets/images/repo-6c4613e5b0b1f97718922cecdb42433b.png)

### Connect your repository and specify it's behaviour

Once you've clicked into the repository, you should see the source code, similar to viewing the repository on GitHub. We'll now click the "Add a Service" button as shown in red below:

![](https://docs.oblivious.ai/assets/images/service-e8b3aa32c71d97b08664525646646b90.png)

You'll now be asked to fill in a form that specifies the services behaviour. For this simple `Hello World` application, we will be using the following configuration:

In [ ]:
Base Image: ubuntu_18_04_proxy_python_3_8

# How you will authenticate the users
Auth Name: default
Auth Type: signed_headers

# At least 1 role for users
Role Name: querier
Role Description: The people who will be greeted
Role Cardinality: 1
Role Auth: default

# List the paths available and who can access them (we'll only use 1 root path)
Path: /
Access: querier
Short Description: Responds "Hello" and their name

# Build Arguments (we don't have any for this application)

# Traffic
Port: 80
Name: inbound

# Meta
Author: Example User
Email: example@oblivious.ai
git: https://github.com/ObliviousAI/my-first-oblivious-app.git
version: 0.1.0

Once filled in, simply hit `Submit & Create`. If there is a validation error the form will let you know what it is. When successfully submitted, you'll see the new service in the Service tab and you'll notice a `.oblivious` folder created in your repository.

### Add a slightly modified Dockerfile & service executables to launch at runtime​

As explained here, enclaves are complex virtual mchines, not typical docker containers. As such, we've put in a number of fixed services which support you application to run inside the enclave. However, this requires slight modifications to how your Dockerfile should be layed out.

First we need to copy your Dockerfile inside of the `.oblivious` folder and remove the line with CMD or Entrypoint



In [ ]:
WORKDIR /code
 
COPY ./requirements.txt /code/requirements.txt
 
RUN pip install --no-cache-dir --upgrade -r /code/requirements.txt
 
COPY ./ /code/

The `FROM` line will be replaced with a varient of the base image we selected in the previous step.

Next, we need the enclave to know how to actually run our service. Internally, this is managed by the s6 service manager, so it requires us to create an executable for each sub-service (program) we want to run. These are placed in a new folder called `.oblivious/services/<service neme>`. Let's add our service nowby creating a folder called `.oblivious/services/server/` and adding the following file named run in it (note no .sh or equivilent extension):

In [ ]:
#!/bin/sh
python /code/uvicorn_serve.py

One of the benefits of using s6 is that if our server were to clash, it will be gracefully restarted inside the enclave with limitted downtime. When these steps are complete, push your code to GitHub so the changes can be updated.

Your repository should have the following structure:

```
├── .oblivious
│   ├── Dockerfile
│   ├── service.yaml
│   └── services
│       └── server
│           └── run
├── api.py 
├── requirements.txt 
├── Dockerfile 
├── README.md 
├── .gitattributes 
└── .gitignore 
```

You've now built a server, given it all of the meta data to be a service and are ready to launch it. All that we need to do from here is to ask our user for their preshared key and to launch the enclave. The steps below are:
1. User creates a public private key pair and shares with the enclave provider.
2. The enclave is launched.
3. User connects to the enclave.

## User creates a public private key pair

Users can download the enclave proxy from [here](https://docs.oblivious.ai/cli/binaries). Once downloaded for their appropriate operating system, they can generate a public private key pair as described [here](https://docs.oblivious.ai/cli/key_generation). The `Public key in base64 format` string is what they will share with the enclave provider:

```bash
./oblv_keygen_gnu_mac test
Generated private key in location: 
<path>/test_private.der
Generated public key in location: 
<path>/test_public.der
Public key in base64 format: 
MIIBCgKCAQEAyRJghHhr6gmkc/mACQz6no8IfVvR/ngZNegJLlJDN9YHOyvBmErcBFmf8yOVK8u5I4iAJCl/GJCgZ9HMMTzkcCVtIj0F/vvVLpPiAs0n9siEiHqBcUIePhD11M4xOIxz/auwPvysxW2EJ79WNr7BSVh/4PsbEmvBBCEnwj8f2bOZ7hOnqv0wsvcBaYZlOJqf8rADZUIH45/KNOmdOMuhwg3AKvohn2lYPtqLLq2G44eSjbB9Xn09SWWlWZKKgor5FypRAdcO79W1ZCvUT4auLajGiXeLyOKfgJdy9e4FnX8F1qsSdA84IN9tlG88qtTsMK41jkDxOVrqCPsyGttGAQIDAQAB%
```


Note: You may need to chmod the executable before you can run it, eg `chmod 755 oblv_keygen_gnu_mac` on macOS.


## The enclave is launched

We are at the final step to launch the enclave. In the console, on the Services tab, search and find your `my-first-oblivious-app` service and hit the launch buttom as hightlighted:

![Service Console Screenshot](https://docs.oblivious.ai/assets/images/service_console-f21ebc36c3dbdc9df05c03a834aaa4bd.png)

We need to select some final build arguments and details of in which region we would like to launch the service. In the first form we can simply specify:

```text
Deployment Name: my-first-deployment
Region Name: eu-west-2, Europe (London)
Visibility Type: Private (so only you can see it in your deployments)
Environment Type: Production
Tags: Test
```

When you confirm the above and move to the next page of the form, simply add the user name to the querier and add their provided base64 Public key (we can use the smallest/cheapest infrastructure option):

```text
user_name: Mr & Mrs Test User
public key: MIIBCgKCAQEAyRJghHhr6gmkc/mACQz6no8IfVvR/ngZNegJLlJDN9YHOyvBmErcBFmf8yOVK8u5I4iAJCl/GJCgZ9HMMTzkcCVtIj0F/vvVLpPiAs0n9siEiHqBcUIePhD11M4xOIxz/auwPvysxW2EJ79WNr7BSVh/4PsbEmvBBCEnwj8f2bOZ7hOnqv0wsvcBaYZlOJqf8rADZUIH45/KNOmdOMuhwg3AKvohn2lYPtqLLq2G44eSjbB9Xn09SWWlWZKKgor5FypRAdcO79W1ZCvUT4auLajGiXeLyOKfgJdy9e4FnX8F1qsSdA84IN9tlG88qtTsMK41jkDxOVrqCPsyGttGAQIDAQAB%
infra-reqs: CPU:4 RAM: 8GM
```

And you are done - hit launch and sit back while your infrastructure is being set up. This can take approximately 5 minutes.

## User connects to the enclave

From the Deployments tab, you should now be able to see your service. Click on the share icon as highlighted below: 

![Share Screenshot](https://docs.oblivious.ai/assets/images/share-fe15d64be8a2f25135b72abed8c343ee.png)

You can share with others and yourself by adding thier usernames and roles accordingly. Everyone who you share the service with will see the details in the Find & Connect tab. 

Note: Sharing an enclave with users does not give them access to the enclave service, it simply makes it visible in their `Find & Connect` tab. Users still have to connect to the enclave with their public/private key pair or alternative credentials.

![Find & Connect Screenshot](https://docs.oblivious.ai/assets/images/find_connect_detail-a783527cc8b6f229a918c6ff88547342.png)

By clicking on the `Connect` button in the `Find & Connect` tab, we see details of the enclave service currently running. In the steps to connect, it specifies how to connect to the service using the `oblv` proxy cli. All the user has to do is to replace the public private key pair deatils with the keys they previously created. 

```bash
oblv connect \
--pcr0 7ae48a6f5643cd31c9dd8e6f7557e16439efb33f78538ac0760a57c7c36bfb0979044a6df15e308e6bb21ee4187f4c7e \
--pcr1 5c01976a546ec6b740353189afd3bf5fe29df96328887111e7c802cf2ff5ad636deed2ab8254e7a51a45fca01d0ae062 \
--pcr2 05073f36c36df2ac8bc796ef1f1fcd97f2965aaeea71db56e6dfe6d6cfe51b75e3d10d3b4087cf4589fdc98d03bf4781 \
--private-key "<path to keys>/test_private.der" \
--public-key "<path to keys>/test_public.der" \
--url https://oblvd-appli-1aaoqoru1rg8j-986428053.enclave.oblivious.ai \
--port 443 --lport 3030
```

The `lport` flag allows you to use your prefered local host port to proxy trafic to and from the enclave. From here we can send and recieve traffic via `localhost:3030` as if the enclave service was running on the local machine.